In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import ipaddress

In [2]:
csv_files = [file for file in os.listdir('data') if file.endswith('.csv')]

df_20250613 = pd.read_csv(os.path.join('data', 'ipinfo_privacy.20250613.csv'))
df_20250923 = pd.read_csv(os.path.join('data', 'ipinfo_privacy.20250923.csv'))

print("DataFrame 20250613 shape:", df_20250613.shape)
print("DataFrame 20250923 shape:", df_20250923.shape)

print(df_20250613.head())
print(df_20250923.head())


DataFrame 20250613 shape: (20118676, 7)
DataFrame 20250923 shape: (20394284, 7)
      network  hosting  proxy    tor  relay    vpn service
0     1.0.0.0     True  False  False  False  False     NaN
1     1.0.0.1     True  False  False  False   True     NaN
2     1.0.0.2     True  False  False  False   True     NaN
3     1.0.0.3     True  False  False  False  False     NaN
4  1.0.0.4/30     True  False  False  False  False     NaN
      network  hosting  proxy    tor  relay    vpn service
0     1.0.0.0     True  False  False  False  False     NaN
1     1.0.0.1     True  False  False  False   True     NaN
2     1.0.0.2     True  False  False  False   True     NaN
3     1.0.0.3     True  False  False  False  False     NaN
4  1.0.0.4/30     True  False  False  False  False     NaN


In [3]:
df_20250613.info()
df_20250923.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20118676 entries, 0 to 20118675
Data columns (total 7 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   network  object
 1   hosting  bool  
 2   proxy    bool  
 3   tor      bool  
 4   relay    bool  
 5   vpn      bool  
 6   service  object
dtypes: bool(5), object(2)
memory usage: 402.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20394284 entries, 0 to 20394283
Data columns (total 7 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   network  object
 1   hosting  bool  
 2   proxy    bool  
 3   tor      bool  
 4   relay    bool  
 5   vpn      bool  
 6   service  object
dtypes: bool(5), object(2)
memory usage: 408.4+ MB


In [4]:
def subnet_count(df):
    return df['network'].str.contains('/').sum()

subnet_count_20250613 = subnet_count(df_20250613)
print("20250613:")
print(f"-- Count: {subnet_count_20250613}")
print(f"-- Proportion: {subnet_count_20250613 / len(df_20250613)}")

subnet_count_20250923 = subnet_count(df_20250923)
print("20250923:")
print(f"-- Count: {subnet_count_20250923}")
print(f"-- Proportion: {subnet_count_20250923 / len(df_20250923)}")

20250613:
-- Count: 8692856
-- Proportion: 0.43207893004489956
20250923:
-- Count: 8655345
-- Proportion: 0.424400533012093


In [9]:
# --- Optimization: Set 'network' as the index ---
# This is the most crucial step for performance. Index-based operations
# are significantly faster than column-based merges on large datasets.
df_20250613_indexed = df_20250613.set_index('network')
df_20250923_indexed = df_20250923.set_index('network')

print(f"Indexed DataFrame 20250613 shape: {df_20250613_indexed.shape}")
print(f"Indexed DataFrame 20250923 shape: {df_20250923_indexed.shape}")


Indexed DataFrame 20250613 shape: (20118676, 7)
Indexed DataFrame 20250923 shape: (20394284, 7)


In [10]:
# --- Split datasets by IP version (IPv4 vs IPv6) ---
print("=== SPLITTING DATASETS BY IP VERSION ===")

def classify_ip_version(network_str):
    """Classify network as IPv4 or IPv6"""
    try:
        # Parse the network (handles both individual IPs and CIDR notation)
        network = ipaddress.ip_network(network_str, strict=False)
        return 4 if network.version == 4 else 6
    except (ipaddress.AddressValueError, ValueError):
        # If parsing fails, try to infer from format
        if ':' in network_str:
            return 6  # Likely IPv6
        elif '.' in network_str:
            return 4  # Likely IPv4
        else:
            return None  # Unknown

def normalize_ipv6_to_64(network_str):
    """Normalize IPv6 addresses/networks to /64 prefixes"""
    try:
        # Parse as network first
        network = ipaddress.ip_network(network_str, strict=False)
        if network.version == 6:
            # Get the /64 prefix (top 64 bits)
            prefix_64 = network.supernet(new_prefix=64)
            return str(prefix_64)
        else:
            return network_str  # Return as-is for IPv4
    except (ipaddress.AddressValueError, ValueError):
        return network_str  # Return as-is if parsing fails

print("Classifying IP versions for 2025-06-13 dataset...")
df_20250613['ip_version'] = df_20250613['network'].apply(classify_ip_version)

print("Classifying IP versions for 2025-09-23 dataset...")
df_20250923['ip_version'] = df_20250923['network'].apply(classify_ip_version)

# Show version distribution
print(f"\n2025-06-13 IP Version Distribution:")
print(df_20250613['ip_version'].value_counts().sort_index())

print(f"\n2025-09-23 IP Version Distribution:")
print(df_20250923['ip_version'].value_counts().sort_index())

# Split datasets by IP version
print(f"\n=== CREATING SEPARATE IPV4 AND IPV6 DATASETS ===")

# IPv4 datasets - keep original networks
df_20250613_ipv4 = df_20250613[df_20250613['ip_version'] == 4].copy()
df_20250923_ipv4 = df_20250923[df_20250923['ip_version'] == 4].copy()

# IPv6 datasets - normalize to /64 prefixes
df_20250613_ipv6 = df_20250613[df_20250613['ip_version'] == 6].copy()
df_20250923_ipv6 = df_20250923[df_20250923['ip_version'] == 6].copy()

print("Normalizing IPv6 networks to /64 prefixes...")
df_20250613_ipv6['network_normalized'] = df_20250613_ipv6['network'].apply(normalize_ipv6_to_64)
df_20250923_ipv6['network_normalized'] = df_20250923_ipv6['network'].apply(normalize_ipv6_to_64)

print(f"IPv4 datasets (original networks):")
print(f"- 2025-06-13: {len(df_20250613_ipv4):,} networks")
print(f"- 2025-09-23: {len(df_20250923_ipv4):,} networks")

print(f"\nIPv6 datasets (before /64 normalization):")
print(f"- 2025-06-13: {len(df_20250613_ipv6):,} networks") 
print(f"- 2025-09-23: {len(df_20250923_ipv6):,} networks")

# Aggregate IPv6 data by /64 prefixes
print(f"\nAggregating IPv6 data by /64 prefixes...")

def aggregate_ipv6_by_prefix(df):
    """Aggregate IPv6 data by /64 prefixes using logical OR for boolean columns"""
    # Group by normalized network and aggregate
    bool_cols = ['hosting', 'proxy', 'tor', 'relay', 'vpn']
    agg_dict = {col: 'max' for col in bool_cols}  # max is equivalent to logical OR for booleans
    agg_dict['service'] = 'first'  # Take first service value
    agg_dict['ip_version'] = 'first'  # Keep IP version
    
    aggregated = df.groupby('network_normalized').agg(agg_dict).reset_index()
    aggregated.rename(columns={'network_normalized': 'network'}, inplace=True)
    return aggregated

df_20250613_ipv6_agg = aggregate_ipv6_by_prefix(df_20250613_ipv6)
df_20250923_ipv6_agg = aggregate_ipv6_by_prefix(df_20250923_ipv6)

print(f"IPv6 datasets (after /64 aggregation):")
print(f"- 2025-06-13: {len(df_20250613_ipv6_agg):,} /64 prefixes") 
print(f"- 2025-09-23: {len(df_20250923_ipv6_agg):,} /64 prefixes")

# Set network as index for each split dataset
df_20250613_ipv4_indexed = df_20250613_ipv4.set_index('network')
df_20250923_ipv4_indexed = df_20250923_ipv4.set_index('network')
df_20250613_ipv6_indexed = df_20250613_ipv6_agg.set_index('network')
df_20250923_ipv6_indexed = df_20250923_ipv6_agg.set_index('network')

print(f"\nIndexed datasets created successfully!")
print(f"IPv4: Using full IP addresses/networks")
print(f"IPv6: Using /64 prefixes with aggregated service classifications")


=== SPLITTING DATASETS BY IP VERSION ===
Classifying IP versions for 2025-06-13 dataset...
Classifying IP versions for 2025-09-23 dataset...

2025-06-13 IP Version Distribution:
ip_version
4    16450714
6     3667962
Name: count, dtype: int64

2025-09-23 IP Version Distribution:
ip_version
4    16899121
6     3495163
Name: count, dtype: int64

=== CREATING SEPARATE IPV4 AND IPV6 DATASETS ===
Normalizing IPv6 networks to /64 prefixes...
IPv4 datasets (original networks):
- 2025-06-13: 16,450,714 networks
- 2025-09-23: 16,899,121 networks

IPv6 datasets (before /64 normalization):
- 2025-06-13: 3,667,962 networks
- 2025-09-23: 3,495,163 networks

Aggregating IPv6 data by /64 prefixes...
IPv6 datasets (after /64 aggregation):
- 2025-06-13: 226,166 /64 prefixes
- 2025-09-23: 253,807 /64 prefixes

Indexed datasets created successfully!
IPv4: Using full IP addresses/networks
IPv6: Using /64 prefixes with aggregated service classifications


In [11]:
# --- Network Alignment Check for IPv4 and IPv6 separately ---
print("=== NETWORK ALIGNMENT CHECK - IPv4 vs IPv6 ===")

def analyze_network_alignment(df_old, df_new, ip_version):
    """Analyze network alignment for a specific IP version"""
    print(f"\n--- {ip_version} Network Alignment ---")
    
    networks_old = set(df_old.index)
    networks_new = set(df_new.index)
    
    print(f"Unique {ip_version} networks in 2025-06-13: {len(networks_old):,}")
    print(f"Unique {ip_version} networks in 2025-09-23: {len(networks_new):,}")
    
    # Check for differences
    missing_in_new = networks_old - networks_new
    missing_in_old = networks_new - networks_old
    common_networks = networks_old & networks_new
    
    print(f"\n{ip_version} Network Alignment:")
    print(f"- Networks only in old dataset: {len(missing_in_new):,}")
    print(f"- Networks only in new dataset: {len(missing_in_old):,}")
    print(f"- Common networks: {len(common_networks):,}")
    
    if len(missing_in_new) > 0:
        print(f"\nSample {ip_version} missing in new: {list(missing_in_new)[:5]}")
    if len(missing_in_old) > 0:
        print(f"Sample {ip_version} missing in old: {list(missing_in_old)[:5]}")
    
    return common_networks

# Analyze IPv4 networks
ipv4_common_networks = analyze_network_alignment(df_20250613_ipv4_indexed, df_20250923_ipv4_indexed, "IPv4")

# Analyze IPv6 networks  
ipv6_common_networks = analyze_network_alignment(df_20250613_ipv6_indexed, df_20250923_ipv6_indexed, "IPv6")

print(f"\n=== SUMMARY ===")
print(f"Total common IPv4 networks: {len(ipv4_common_networks):,}")
print(f"Total common IPv6 networks: {len(ipv6_common_networks):,}")
print(f"Total common networks: {len(ipv4_common_networks) + len(ipv6_common_networks):,}")

# Create aligned dataframes for IPv4
if len(ipv4_common_networks) > 0:
    df_old_ipv4_aligned = df_20250613_ipv4_indexed.loc[list(ipv4_common_networks)].sort_index()
    df_new_ipv4_aligned = df_20250923_ipv4_indexed.loc[list(ipv4_common_networks)].sort_index()
    print(f"IPv4 aligned datasets shape: {df_old_ipv4_aligned.shape}")

# Create aligned dataframes for IPv6
if len(ipv6_common_networks) > 0:
    df_old_ipv6_aligned = df_20250613_ipv6_indexed.loc[list(ipv6_common_networks)].sort_index()
    df_new_ipv6_aligned = df_20250923_ipv6_indexed.loc[list(ipv6_common_networks)].sort_index()
    print(f"IPv6 aligned datasets shape: {df_old_ipv6_aligned.shape}")


=== NETWORK ALIGNMENT CHECK - IPv4 vs IPv6 ===

--- IPv4 Network Alignment ---
Unique IPv4 networks in 2025-06-13: 16,450,714
Unique IPv4 networks in 2025-09-23: 16,899,121

IPv4 Network Alignment:
- Networks only in old dataset: 7,773,757
- Networks only in new dataset: 8,222,164
- Common networks: 8,676,957

Sample IPv4 missing in new: ['111.252.228.106', '109.170.146.171', '113.2.134.115', '115.235.79.82', '221.203.149.104']
Sample IPv4 missing in old: ['189.46.14.245', '119.200.97.165', '50.151.4.10', '121.61.48.24', '13.58.14.88/30']

--- IPv6 Network Alignment ---
Unique IPv6 networks in 2025-06-13: 226,166
Unique IPv6 networks in 2025-09-23: 253,807

IPv6 Network Alignment:
- Networks only in old dataset: 21,565
- Networks only in new dataset: 49,206
- Common networks: 204,601

Sample IPv6 missing in new: ['2a01:4f8:10a:c00::/56', '2401:1770::/28', '2001:67c:9e0::/48', '2a00:1630:2:1e00::/55', '2a05:dfc6:9300::/40']
Sample IPv6 missing in old: ['2a01:7a00:13:1000::/52', '2606:40

In [12]:
# --- IPv4 Service Classification Churn Analysis ---
print("=== IPv4 SERVICE CLASSIFICATION CHURN ANALYSIS ===")

if len(ipv4_common_networks) > 0:
    # Service columns to analyze
    service_columns = ['hosting', 'proxy', 'tor', 'relay', 'vpn']
    
    # Track networks that changed for each service
    ipv4_service_changes = {}
    ipv4_total_changed_networks = set()
    
    for service in service_columns:
        # Find networks where the service classification changed
        old_values = df_old_ipv4_aligned[service]
        new_values = df_new_ipv4_aligned[service]
        
        # Networks that changed (True->False or False->True)
        changed_mask = old_values != new_values
        changed_networks = df_old_ipv4_aligned[changed_mask].index.tolist()
        
        # Break down into specific change types
        gained_service = df_old_ipv4_aligned[(old_values == False) & (new_values == True)].index.tolist()
        lost_service = df_old_ipv4_aligned[(old_values == True) & (new_values == False)].index.tolist()
        
        ipv4_service_changes[service] = {
            'total_changed': len(changed_networks),
            'gained': len(gained_service),
            'lost': len(lost_service),
            'gained_networks': gained_service[:10],  # Sample
            'lost_networks': lost_service[:10]       # Sample
        }
        
        ipv4_total_changed_networks.update(changed_networks)
        
        print(f"\nIPv4 {service.upper()} Classification Changes:")
        print(f"- Total networks changed: {len(changed_networks):,}")
        print(f"- Gained {service}: {len(gained_service):,}")
        print(f"- Lost {service}: {len(lost_service):,}")
        print(f"- Churn rate: {len(changed_networks)/len(ipv4_common_networks)*100:.4f}%")
    
    print(f"\n=== IPv4 OVERALL CLASSIFICATION CHURN ===")
    print(f"- IPv4 networks with any service change: {len(ipv4_total_changed_networks):,}")
    print(f"- IPv4 overall churn rate: {len(ipv4_total_changed_networks)/len(ipv4_common_networks)*100:.4f}%")
    print(f"- IPv4 stable networks (no changes): {len(ipv4_common_networks) - len(ipv4_total_changed_networks):,}")
    print(f"- IPv4 stability rate: {(len(ipv4_common_networks) - len(ipv4_total_changed_networks))/len(ipv4_common_networks)*100:.4f}%")
else:
    print("No common IPv4 networks found for analysis.")


=== IPv4 SERVICE CLASSIFICATION CHURN ANALYSIS ===

IPv4 HOSTING Classification Changes:
- Total networks changed: 121,638
- Gained hosting: 51,851
- Lost hosting: 69,787
- Churn rate: 1.4019%

IPv4 PROXY Classification Changes:
- Total networks changed: 1,136
- Gained proxy: 371
- Lost proxy: 765
- Churn rate: 0.0131%

IPv4 TOR Classification Changes:
- Total networks changed: 39
- Gained tor: 10
- Lost tor: 29
- Churn rate: 0.0004%

IPv4 RELAY Classification Changes:
- Total networks changed: 11
- Gained relay: 11
- Lost relay: 0
- Churn rate: 0.0001%

IPv4 VPN Classification Changes:
- Total networks changed: 18,257
- Gained vpn: 9,188
- Lost vpn: 9,069
- Churn rate: 0.2104%

=== IPv4 OVERALL CLASSIFICATION CHURN ===
- IPv4 networks with any service change: 140,058
- IPv4 overall churn rate: 1.6141%
- IPv4 stable networks (no changes): 8,536,899
- IPv4 stability rate: 98.3859%


In [13]:
# --- IPv6 Service Classification Churn Analysis ---
print("=== IPv6 SERVICE CLASSIFICATION CHURN ANALYSIS ===")

if len(ipv6_common_networks) > 0:
    # Track networks that changed for each service
    ipv6_service_changes = {}
    ipv6_total_changed_networks = set()
    
    for service in service_columns:
        # Find networks where the service classification changed
        old_values = df_old_ipv6_aligned[service]
        new_values = df_new_ipv6_aligned[service]
        
        # Networks that changed (True->False or False->True)
        changed_mask = old_values != new_values
        changed_networks = df_old_ipv6_aligned[changed_mask].index.tolist()
        
        # Break down into specific change types
        gained_service = df_old_ipv6_aligned[(old_values == False) & (new_values == True)].index.tolist()
        lost_service = df_old_ipv6_aligned[(old_values == True) & (new_values == False)].index.tolist()
        
        ipv6_service_changes[service] = {
            'total_changed': len(changed_networks),
            'gained': len(gained_service),
            'lost': len(lost_service),
            'gained_networks': gained_service[:10],  # Sample
            'lost_networks': lost_service[:10]       # Sample
        }
        
        ipv6_total_changed_networks.update(changed_networks)
        
        print(f"\nIPv6 {service.upper()} Classification Changes:")
        print(f"- Total networks changed: {len(changed_networks):,}")
        print(f"- Gained {service}: {len(gained_service):,}")
        print(f"- Lost {service}: {len(lost_service):,}")
        print(f"- Churn rate: {len(changed_networks)/len(ipv6_common_networks)*100:.4f}%")
    
    print(f"\n=== IPv6 OVERALL CLASSIFICATION CHURN ===")
    print(f"- IPv6 networks with any service change: {len(ipv6_total_changed_networks):,}")
    print(f"- IPv6 overall churn rate: {len(ipv6_total_changed_networks)/len(ipv6_common_networks)*100:.4f}%")
    print(f"- IPv6 stable networks (no changes): {len(ipv6_common_networks) - len(ipv6_total_changed_networks):,}")
    print(f"- IPv6 stability rate: {(len(ipv6_common_networks) - len(ipv6_total_changed_networks))/len(ipv6_common_networks)*100:.4f}%")
else:
    print("No common IPv6 networks found for analysis.")


=== IPv6 SERVICE CLASSIFICATION CHURN ANALYSIS ===

IPv6 HOSTING Classification Changes:
- Total networks changed: 65
- Gained hosting: 61
- Lost hosting: 4
- Churn rate: 0.0318%

IPv6 PROXY Classification Changes:
- Total networks changed: 8
- Gained proxy: 2
- Lost proxy: 6
- Churn rate: 0.0039%

IPv6 TOR Classification Changes:
- Total networks changed: 5
- Gained tor: 1
- Lost tor: 4
- Churn rate: 0.0024%

IPv6 RELAY Classification Changes:
- Total networks changed: 60
- Gained relay: 60
- Lost relay: 0
- Churn rate: 0.0293%

IPv6 VPN Classification Changes:
- Total networks changed: 8
- Gained vpn: 7
- Lost vpn: 1
- Churn rate: 0.0039%

=== IPv6 OVERALL CLASSIFICATION CHURN ===
- IPv6 networks with any service change: 144
- IPv6 overall churn rate: 0.0704%
- IPv6 stable networks (no changes): 204,457
- IPv6 stability rate: 99.9296%
